# This code is to get the labeled_energy_data csv file of each model

## Import the lib

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numba as nb

## load function

### interpolate function

In [2]:
@nb.jit(nopython=True)
def interpolate_point(times, powers, target_time):
    n = len(times)
    if n == 0:
        return 0.0
    if target_time <= times[0]:
        return powers[0]
    if target_time >= times[-1]:
        return powers[-1]
    
    # Binary search
    left, right = 0, n-1
    while left <= right:
        mid = (left + right) // 2
        if times[mid] == target_time:
            return powers[mid]
        elif times[mid] < target_time:
            left = mid + 1
        else:
            right = mid - 1
            
    # Linear interpolation
    pos = left
    t1, p1 = times[pos-1], powers[pos-1]
    t2, p2 = times[pos], powers[pos]
    ratio = (target_time - t1) / (t2 - t1)
    return p1 + (p2 - p1) * ratio

@nb.jit(nopython=True)
def integrate_power_over_interval(samples, start_time, end_time):
    times = samples[:, 0]
    powers = samples[:, 1]
    
    # Get start and end powers through interpolation
    start_power = interpolate_point(times, powers, start_time)
    end_power = interpolate_point(times, powers, end_time)
    
    # Filter points within interval
    mask = (times >= start_time) & (times <= end_time)
    interval_times = times[mask]
    interval_powers = powers[mask]
    
    # Create array including boundary points
    n_points = len(interval_times)
    full_times = np.zeros(n_points + 2)
    full_powers = np.zeros(n_points + 2)
    
    # Add boundary points
    full_times[0] = start_time
    full_powers[0] = start_power
    full_times[-1] = end_time
    full_powers[-1] = end_power
    
    # Add interior points
    if n_points > 0:
        full_times[1:-1] = interval_times
        full_powers[1:-1] = interval_powers
    
    # Integration using trapezoidal rule
    total_energy = 0.0
    for i in range(len(full_times)-1):
        dt = full_times[i+1] - full_times[i]
        avg_p = (full_powers[i] + full_powers[i+1]) / 2.0
        total_energy += avg_p * dt
        
    return total_energy


### label energy function

In [3]:
def label_energy_consumption(energy_data, to_device, forward, loss, backward, optimize):
    # Create a copy of the energy_data dataframe to avoid modifying the original
    labeled_energy_data = energy_data.copy()
    
    # Initialize a new column for the step labels
    labeled_energy_data['step'] = 'idle'
    
    # Define a helper function to label the steps
    def label_steps(energy_data, step_energy, step_name):
        for epoch in range(step_energy.shape[0]):
            for batch in range(step_energy.shape[1]):
                start_time = step_energy[epoch][batch][0]
                end_time = step_energy[epoch][batch][1]
                mask = (energy_data['timestamp'] >= start_time) & (energy_data['timestamp'] <= end_time)
                labeled_energy_data.loc[mask, 'step'] = step_name
    
    # Label each step
    label_steps(labeled_energy_data, to_device, 'to_device')
    label_steps(labeled_energy_data, forward, 'forward')
    label_steps(labeled_energy_data, loss, 'loss')
    label_steps(labeled_energy_data, backward, 'backward')
    label_steps(labeled_energy_data, optimize, 'optimize')
    
    return labeled_energy_data

### load the label function

## find the data path

In [4]:
current_path = os.getcwd()
data_path = os.path.join(current_path, 'ModelsData')
data_folders = os.listdir(data_path)
print(data_folders)

['googlenet_origin', 'resnet50', 'resnet34', 'mobilenetv2', 'vgg13', 'googlenet_mod7', 'googlenet_mod9', 'googlenet_mod8', 'googlenet_mod1', 'googlenet_mod6', 'resnet18', 'mobilenetv1', 'vgg16', 'vgg11', 'googlenet_mod3', 'googlenet_mod4', 'googlenet_mod5', 'googlenet_mod2', 'alexnet']


## load all the model data

In [5]:
# set the epoch number, batch size and the round number as well as the sampling rate
epoch = 5
batch_size = 128
round_num = 0
sampling_rate = 2

In [6]:
# # for all the folder names in the data folder, generate the path to the folder
# # and print the path
# folder_name = f'E{epoch}_B{batch_size}_R{round_num}_SR{sampling_rate}_layer'

# data_dir = 'fashion_mnist'
# # data_dir = 'cifar100'
# # data_dir = 'cifar10'



# print(folder_name)
# for folder in data_folders:
#     folder_path = os.path.join(data_path, folder)
#     folder_path = os.path.join(folder_path, folder_name)
#     folder_path = os.path.join(folder_path, data_dir)
#     print(folder_path)

#     # load the csv files 
#     energy_data = pd.read_csv(os.path.join(folder_path, 'energy_consumption_file.csv'))

#     # load the npy files
#     to_device = np.load(os.path.join(folder_path, 'to_device.npy'), allow_pickle=True)
#     forward = np.load(os.path.join(folder_path, 'forward.npy'), allow_pickle=True)
#     loss = np.load(os.path.join(folder_path, 'loss.npy'), allow_pickle=True)
#     backward = np.load(os.path.join(folder_path, 'backward.npy'), allow_pickle=True)
#     optimize = np.load(os.path.join(folder_path, 'optimize.npy'), allow_pickle=True)

#     # Set the display format for floating-point numbers to avoid scientific notation
#     pd.options.display.float_format = '{:.6f}'.format

#     # Use the function to label the energy consumption data
#     labeled_energy_data = label_energy_consumption(energy_data, to_device, forward, loss, backward, optimize)
#     print(labeled_energy_data.head())

#     # save the file to the folder
#     labeled_energy_data.to_csv(os.path.join(folder_path, 'labeled_energy_data.csv'), index=False)

In [14]:
# for all the folder names in the data folder, generate the path to the folder
# Find the index of 'alexnet' in data_folders
alexnet_index = data_folders.index('alexnet')
print(f"Index of 'alexnet': {alexnet_index}")

# and print the path
folder_name = f'E{epoch}_B{batch_size}_R{round_num}_SR{sampling_rate}_layer'

data_dir = 'fashion_mnist'
# data_dir = 'cifar100'
# data_dir = 'cifar10'
print(folder_name)

folder = data_folders[alexnet_index]

folder_path = os.path.join(data_path, folder)
folder_path = os.path.join(folder_path, folder_name)
folder_path = os.path.join(folder_path, data_dir)
# print(folder_path)

print(folder_path)



# load the csv files 
energy_data = pd.read_csv(os.path.join(folder_path, 'energy_consumption_file.csv'))
forward_layer_data = pd.read_csv(os.path.join(folder_path, 'layer_time.csv'))

# load the npy files
to_device = np.load(os.path.join(folder_path, 'to_device.npy'), allow_pickle=True)
forward = np.load(os.path.join(folder_path, 'forward.npy'), allow_pickle=True)
loss = np.load(os.path.join(folder_path, 'loss.npy'), allow_pickle=True)
backward = np.load(os.path.join(folder_path, 'backward.npy'), allow_pickle=True)
optimize = np.load(os.path.join(folder_path, 'optimize.npy'), allow_pickle=True)

# Set the display format for floating-point numbers to avoid scientific notation
pd.options.display.float_format = '{:.6f}'.format

print(forward_layer_data.head())
print(forward_layer_data.shape)

# # Use the function to label the energy consumption data
# labeled_energy_data = label_energy_consumption(energy_data, to_device, forward, loss, backward, optimize)
# print(labeled_energy_data.head())

    # # save the file to the folder
    # labeled_energy_data.to_csv(os.path.join(folder_path, 'labeled_energy_data.csv'), index=False)

Index of 'alexnet': 18
E5_B128_R0_SR2_layer
/Users/dtjgp/Projects/GreenAI/3080/ModelsData/alexnet/E5_B128_R0_SR2_layer/fashion_mnist
           Conv2d_0            ReLU_1       MaxPool2d_2          Conv2d_3  \
0 1734613207.836334 1734613207.836997 1734613207.837243 1734613207.837460   
1 1734613207.836994 1734613207.837240 1734613207.837458 1734613207.839777   

             ReLU_4       MaxPool2d_5          Conv2d_6            ReLU_7  \
0 1734613207.839780 1734613207.839961 1734613207.840120 1734613207.841113   
1 1734613207.839959 1734613207.840118 1734613207.841111 1734613207.841207   

           Conv2d_8            ReLU_9  ...      MaxPool2d_12  \
0 1734613207.841209 1734613207.842361  ... 1734613207.843324   
1 1734613207.842359 1734613207.842434  ... 1734613207.843395   

   AdaptiveAvgPool2d_13        Flatten_14         Linear_15           ReLU_16  \
0     1734613207.843396 1734613207.843550 1734613207.843574 1734613207.844162   
1     1734613207.843548 1734613207.843573 173461

,Conv2d_0,ReLU_1,MaxPool2d_2,Conv2d_3,ReLU_4,MaxPool2d_5,Conv2d_6,ReLU_7,Conv2d_8,ReLU_9,...,MaxPool2d_12,AdaptiveAvgPool2d_13,Flatten_14,Linear_15,ReLU_16,Dropout_17,Linear_18,ReLU_19,Dropout_20,Linear_21
0,1734613207.836334,1734613207.836997,1734613207.837243,1734613207.837460,1734613207.839780,1734613207.839961,1734613207.840120,1734613207.841113,1734613207.841209,1734613207.842361,...,1734613207.843324,1734613207.843396,1734613207.843550,1734613207.843574,1734613207.844162,1734613207.844198,1734613207.844248,1734613207.844535,1734613207.844568,1734613207.844609
1,1734613207.836994,1734613207.837240,1734613207.837458,1734613207.839777,1734613207.839959,1734613207.840118,1734613207.841111,1734613207.841207,1734613207.842359,1734613207.842434,...,1734613207.843395,1734613207.843548,1734613207.843573,1734613207.844160,1734613207.844197,1734613207.844246,1734613207.844533,1734613207.844567,1734613207.844607,1734613207.844665
